In [47]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.shuffle.useOldFetchProtocol','true'). \
config("spark.sql.warehouse.dir", f"/user/itv021558/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [48]:
loans_schema = 'loan_id string,member_id string,loan_amount float,funded_amount float,loan_term_months string,interest_rate float,monthly_installment float,issue_date string,loan_status string,loan_purpose string,loan_title string'

In [49]:
loans_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loans_schema) \
.load("/public/trendytech/lendingclubproject/raw/loans_data_csv")

In [23]:
loans_raw_df

+--------+--------------------+-----------+-------------+----------------+-------------+-------------------+----------+-----------+------------------+--------------------+
| loan_id|           member_id|loan_amount|funded_amount|loan_term_months|interest_rate|monthly_installment|issue_date|loan_status|      loan_purpose|          loan_title|
+--------+--------------------+-----------+-------------+----------------+-------------+-------------------+----------+-----------+------------------+--------------------+
|56633077|b59d80da191f5b573...|     3000.0|       3000.0|       36 months|         7.89|              93.86|  Aug-2015| Fully Paid|       credit_card|Credit card refin...|
|55927518|202d9f56ecb7c3bc9...|    15600.0|      15600.0|       36 months|         7.89|             488.06|  Aug-2015| Fully Paid|       credit_card|Credit card refin...|
|56473345|e5a140c0922b554b9...|    20000.0|      20000.0|       36 months|         9.17|             637.58|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|56463188|e12aefc548f750777...|    11200.0|      11200.0|       60 months|        21.99|             309.27|  Aug-2015| Fully Paid|  home_improvement|    Home improvement|
|56473316|1b3a50d854fbbf97e...|    16000.0|      16000.0|       60 months|        20.99|             432.77|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|
|56663266|1c4329e5f17697127...|    20000.0|      20000.0|       60 months|        13.33|             458.45|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|
|56483027|5026c86ad983175eb...|    10000.0|      10000.0|       36 months|        12.69|             335.45|  Aug-2015| Fully Paid|             other|               Other|
|56613385|9847d8c1e9d0b2084...|    23400.0|      23400.0|       60 months|        19.19|             609.46|  Aug-2015|    Current|    small_business|            Business|
|56643620|8340dbe1adea41fb4...|    16000.0|      16000.0|       36 months|         5.32|             481.84|  Jul-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|56533114|d4de0de3ab7d79ad4...|    25450.0|      25450.0|       36 months|        27.31|            1043.24|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|
|56984127|3df3cdeddb74a8712...|    24000.0|      24000.0|       36 months|         7.89|             750.86|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|56131038|9dd72636b1b4045b4...|    12000.0|      12000.0|       60 months|        15.61|             289.34|  Aug-2015|    Current|       credit_card|Credit card refin...|
|56452581|1d4e1ef4353b73c00...|    25000.0|      25000.0|       36 months|        13.33|             846.33|  Jul-2015| Fully Paid|  home_improvement|    Home improvement|
|56685321|6f196952e71277fd4...|    35000.0|      35000.0|       60 months|        18.55|             899.28|  Aug-2015|Charged Off|    small_business|            Business|
|56563360|9c617cbc6e3e3d6a1...|    25000.0|      25000.0|       60 months|        21.99|             690.34|  Aug-2015|    Current|       credit_card|Credit card refin...|
|56533113|45eb04dd75400a942...|    20000.0|      20000.0|       36 months|        13.99|             683.46|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|56904118|08d233c81f9e50726...|     9525.0|       9525.0|       36 months|        12.29|             317.69|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|
|56473361|8ff4d0ed17a1cab92...|     4200.0|       4200.0|       36 months|        15.61|             146.86|  Aug-2015| Fully Paid|       credit_card|Credit card refin...|
|56021410|aec13cfd611b2cfea...|     8625.0|       8625.0|       36 months|        16.55|             305.58|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|56473332|9ba8c96ccba6cd1f0...|    19000.0|      19000.0|       60 months|        17.57|             478.05|  Jul-2015|    Current|       credit_card|Credit card refin...|
+--------+--------------------+-----------+-

In [50]:
from pyspark.sql.functions import current_timestamp

In [51]:
loans_ingest = loans_raw_df.withColumn("ingest_date", current_timestamp())

In [52]:
loans_ingest.createOrReplaceTempView("loans")

In [29]:
spark.sql("select count(*) from loans where loan_amount is null")

+--------+
|count(1)|
+--------+
|      33|
+--------+

In [53]:
 columns_to_check = ["loan_amount", "funded_amount", "loan_term_months",
 "interest_rate",
 "monthly_installment","issue_date","loan_status","loan_purpose"]

In [54]:
loans_filtered = loans_ingest.na.drop(subset = columns_to_check)

In [55]:
loans_filtered.createOrReplaceTempView("loans")

In [56]:
from pyspark.sql.functions import regexp_replace, col

In [57]:
loans_term  = loans_filtered \
.withColumn("loan_term_months", (regexp_replace(col("loan_term_months"), " months", "") \
.cast("int") / 12) \
.cast("int")) \
.withColumnRenamed("loan_term_months","loan_term_years")

In [36]:
loans_term

+--------+--------------------+-----------+-------------+---------------+-------------+-------------------+----------+-----------+------------------+--------------------+--------------------+
| loan_id|           member_id|loan_amount|funded_amount|loan_term_years|interest_rate|monthly_installment|issue_date|loan_status|      loan_purpose|          loan_title|         ingest_date|
+--------+--------------------+-----------+-------------+---------------+-------------+-------------------+----------+-----------+------------------+--------------------+--------------------+
|56633077|b59d80da191f5b573...|     3000.0|       3000.0|              3|         7.89|              93.86|  Aug-2015| Fully Paid|       credit_card|Credit card refin...|2025-11-11 01:14:...|
|55927518|202d9f56ecb7c3bc9...|    15600.0|      15600.0|              3|         7.89|             488.06|  Aug-2015| Fully Paid|       credit_card|Credit card refin...|2025-11-11 01:14:...|
|56473345|e5a140c0922b554b9...|    20000.0|      20000.0|              3|         9.17|             637.58|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|2025-11-11 01:14:...|
|56463188|e12aefc548f750777...|    11200.0|      11200.0|              5|        21.99|             309.27|  Aug-2015| Fully Paid|  home_improvement|    Home improvement|2025-11-11 01:14:...|
|56473316|1b3a50d854fbbf97e...|    16000.0|      16000.0|              5|        20.99|             432.77|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|2025-11-11 01:14:...|
|56663266|1c4329e5f17697127...|    20000.0|      20000.0|              5|        13.33|             458.45|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|2025-11-11 01:14:...|
|56483027|5026c86ad983175eb...|    10000.0|      10000.0|              3|        12.69|             335.45|  Aug-2015| Fully Paid|             other|               Other|2025-11-11 01:14:...|
|56613385|9847d8c1e9d0b2084...|    23400.0|      23400.0|              5|        19.19|             609.46|  Aug-2015|    Current|    small_business|            Business|2025-11-11 01:14:...|
|56643620|8340dbe1adea41fb4...|    16000.0|      16000.0|              3|         5.32|             481.84|  Jul-2015| Fully Paid|debt_consolidation|  Debt consolidation|2025-11-11 01:14:...|
|56533114|d4de0de3ab7d79ad4...|    25450.0|      25450.0|              3|        27.31|            1043.24|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|2025-11-11 01:14:...|
|56984127|3df3cdeddb74a8712...|    24000.0|      24000.0|              3|         7.89|             750.86|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|2025-11-11 01:14:...|
|56131038|9dd72636b1b4045b4...|    12000.0|      12000.0|              5|        15.61|             289.34|  Aug-2015|    Current|       credit_card|Credit card refin...|2025-11-11 01:14:...|
|56452581|1d4e1ef4353b73c00...|    25000.0|      25000.0|              3|        13.33|             846.33|  Jul-2015| Fully Paid|  home_improvement|    Home improvement|2025-11-11 01:14:...|
|56685321|6f196952e71277fd4...|    35000.0|      35000.0|              5|        18.55|             899.28|  Aug-2015|Charged Off|    small_business|            Business|2025-11-11 01:14:...|
|56563360|9c617cbc6e3e3d6a1...|    25000.0|      25000.0|              5|        21.99|             690.34|  Aug-2015|    Current|       credit_card|Credit card refin...|2025-11-11 01:14:...|
|56533113|45eb04dd75400a942...|    20000.0|      20000.0|              3|        13.99|             683.46|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|2025-11-11 01:14:...|
|56904118|08d233c81f9e50726...|     9525.0|       9525.0|              3|        12.29|             317.69|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|2025-11-11 01:14:...|
|56473361|8ff4d0ed17a1cab92...|     4200.0|       4200.0|              3|        15.61|             146.86|  Aug-2015| Fully Paid|       credit_card|Credit card

In [58]:
loans_term.createOrReplaceTempView("loans")

In [38]:
spark.sql("select loan_purpose, count(*) as total from loans group by loan_purpose order by total desc")

+--------------------+-------+
|        loan_purpose|  total|
+--------------------+-------+
|  debt_consolidation|1277790|
|         credit_card| 516926|
|    home_improvement| 150440|
|               other| 139413|
|      major_purchase|  50429|
|             medical|  27481|
|      small_business|  24659|
|                 car|  24009|
|            vacation|  15525|
|              moving|  15402|
|               house|  14131|
|             wedding|   2351|
|    renewable_energy|   1445|
|         educational|    412|
|please feel free ...|      1|
|and also pay off ...|      1|
|and it lasts thro...|      1|
|I became his prim...|      1|
|              stocks|      1|
|<br/><br/>Lending...|      1|
+--------------------+-------+
only showing top 20 rows

In [59]:
 loan_purpose_lookup = ["debt_consolidation", "credit_card",
 "home_improvement", "other", "major_purchase", "medical", "small_business",
 "car", "vacation", "moving", "house", "wedding", "renewable_energy",
 "educational"]

In [60]:
from pyspark.sql.functions import when

In [61]:
loan_purpose = loans_term.withColumn("loan_purpose", when(col("loan_purpose").isin(loan_purpose_lookup), col("loan_purpose")).otherwise("other"))

In [62]:
loan_purpose

+--------+--------------------+-----------+-------------+---------------+-------------+-------------------+----------+-----------+------------------+--------------------+--------------------+
| loan_id|           member_id|loan_amount|funded_amount|loan_term_years|interest_rate|monthly_installment|issue_date|loan_status|      loan_purpose|          loan_title|         ingest_date|
+--------+--------------------+-----------+-------------+---------------+-------------+-------------------+----------+-----------+------------------+--------------------+--------------------+
|56633077|b59d80da191f5b573...|     3000.0|       3000.0|              3|         7.89|              93.86|  Aug-2015| Fully Paid|       credit_card|Credit card refin...|2025-11-11 01:26:...|
|55927518|202d9f56ecb7c3bc9...|    15600.0|      15600.0|              3|         7.89|             488.06|  Aug-2015| Fully Paid|       credit_card|Credit card refin...|2025-11-11 01:26:...|
|56473345|e5a140c0922b554b9...|    20000.0|      20000.0|              3|         9.17|             637.58|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|2025-11-11 01:26:...|
|56463188|e12aefc548f750777...|    11200.0|      11200.0|              5|        21.99|             309.27|  Aug-2015| Fully Paid|  home_improvement|    Home improvement|2025-11-11 01:26:...|
|56473316|1b3a50d854fbbf97e...|    16000.0|      16000.0|              5|        20.99|             432.77|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|2025-11-11 01:26:...|
|56663266|1c4329e5f17697127...|    20000.0|      20000.0|              5|        13.33|             458.45|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|2025-11-11 01:26:...|
|56483027|5026c86ad983175eb...|    10000.0|      10000.0|              3|        12.69|             335.45|  Aug-2015| Fully Paid|             other|               Other|2025-11-11 01:26:...|
|56613385|9847d8c1e9d0b2084...|    23400.0|      23400.0|              5|        19.19|             609.46|  Aug-2015|    Current|    small_business|            Business|2025-11-11 01:26:...|
|56643620|8340dbe1adea41fb4...|    16000.0|      16000.0|              3|         5.32|             481.84|  Jul-2015| Fully Paid|debt_consolidation|  Debt consolidation|2025-11-11 01:26:...|
|56533114|d4de0de3ab7d79ad4...|    25450.0|      25450.0|              3|        27.31|            1043.24|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|2025-11-11 01:26:...|
|56984127|3df3cdeddb74a8712...|    24000.0|      24000.0|              3|         7.89|             750.86|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|2025-11-11 01:26:...|
|56131038|9dd72636b1b4045b4...|    12000.0|      12000.0|              5|        15.61|             289.34|  Aug-2015|    Current|       credit_card|Credit card refin...|2025-11-11 01:26:...|
|56452581|1d4e1ef4353b73c00...|    25000.0|      25000.0|              3|        13.33|             846.33|  Jul-2015| Fully Paid|  home_improvement|    Home improvement|2025-11-11 01:26:...|
|56685321|6f196952e71277fd4...|    35000.0|      35000.0|              5|        18.55|             899.28|  Aug-2015|Charged Off|    small_business|            Business|2025-11-11 01:26:...|
|56563360|9c617cbc6e3e3d6a1...|    25000.0|      25000.0|              5|        21.99|             690.34|  Aug-2015|    Current|       credit_card|Credit card refin...|2025-11-11 01:26:...|
|56533113|45eb04dd75400a942...|    20000.0|      20000.0|              3|        13.99|             683.46|  Aug-2015| Fully Paid|debt_consolidation|  Debt consolidation|2025-11-11 01:26:...|
|56904118|08d233c81f9e50726...|     9525.0|       9525.0|              3|        12.29|             317.69|  Aug-2015|Charged Off|debt_consolidation|  Debt consolidation|2025-11-11 01:26:...|
|56473361|8ff4d0ed17a1cab92...|     4200.0|       4200.0|              3|        15.61|             146.86|  Aug-2015| Fully Paid|       credit_card|Credit card

In [63]:
loan_purpose.createOrReplaceTempView("loans")

In [64]:
spark.sql("select loan_purpose, count(*) as total from loans group by loan_purpose order by total desc")

+------------------+-------+
|      loan_purpose|  total|
+------------------+-------+
|debt_consolidation|1277790|
|       credit_card| 516926|
|  home_improvement| 150440|
|             other| 139667|
|    major_purchase|  50429|
|           medical|  27481|
|    small_business|  24659|
|               car|  24009|
|          vacation|  15525|
|            moving|  15402|
|             house|  14131|
|           wedding|   2351|
|  renewable_energy|   1445|
|       educational|    412|
+------------------+-------+

In [66]:
loan_purpose.write \
.option("header", True) \
.mode("overwrite") \
.option("path","/user/itv021558/lendingclubproject/cleaned/loans_parquet") \
.save()

In [67]:
spark.stop()